In [ ]:
%matplotlib qt5
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tabulate import tabulate
import awkward as ak
import numpy as np

import Master
import vector

from prod4a_merge_study import EventSelection, ShowerMergeQuantities, Percentage


events = Master.Data("work/ROOTFiles/Prod4a_6GeV_BeamSim_00_evd.root", True)

# create hit space point arrays
events.recoParticles.spacePoints = ak.zip({"x" : events.io.Get("reco_daughter_allShower_spacePointX"), 
                                           "y" : events.io.Get("reco_daughter_allShower_spacePointY"),
                                           "z" : events.io.Get("reco_daughter_allShower_spacePointZ")})
start_showers = EventSelection(events)

mask = np.logical_or(events.recoParticles.cnnScore > 0.64, np.logical_or(*start_showers)) # select PFOs which pass CNN selection or are start showers
mask = np.logical_or(events.recoParticles.beam_number == events.recoParticles.number, mask)
events.Filter([mask])
start_showers = [start_showers[i][mask] for i in range(2)]


#* get boolean mask of PFP's to merge
to_merge = np.logical_not(np.logical_or(*start_showers))

q = ShowerMergeQuantities(events, to_merge)
q.Evaluate(events, start_showers)

#* get boolean mask of PFP's which are actual fragments of the starting showers
start_shower_ID = events.trueParticlesBT.number[np.logical_or(*start_showers)]
to_merge_ID = events.trueParticlesBT.number[to_merge]
signal = [to_merge_ID == start_shower_ID[:, i] for i in range(2)] # signal are the PFOs which is a fragment of the ith starting shower

#* define signal and background
signal_all = np.logical_or(*signal)
signal_all = signal_all[q.null]
background = np.logical_not(signal_all) # background is all other PFOs unrelated to the pi0 decay
signal = [signal[i][q.null] for i in range(2)]

In [ ]:
nEvents = ak.num(events.recoParticles.spacePoints.x, 0)
eventNum = 5 # 11 and 13 identical??
nPFO = ak.num(events.recoParticles.spacePoints.x)
showSignal = True
showBackground = True

In [ ]:
import prod4a_evd
display = prod4a_evd.EventDisplay(events.eventNum[eventNum], events.run[eventNum], events.subRun[eventNum])
display.xy.set_facecolor("whitesmoke")
display.xz.set_facecolor("whitesmoke")

#* now Plot start showers:
start_showers_merged = np.logical_or(*start_showers)
points = events.recoParticles.spacePoints[start_showers_merged][eventNum]
startPoints = events.recoParticles.startPos[start_showers_merged][eventNum]
directions = events.recoParticles.direction[start_showers_merged][eventNum]
pdgs = events.trueParticlesBT.pdg[start_showers_merged][eventNum]
display.PlotPFO(points[0], marker = "x", colour = "green", startPoint = startPoints[0], direction = directions[0], pdg=pdgs[0])
display.PlotPFO(points[1], marker = "x", colour = "blue", startPoint = startPoints[1], direction = directions[1], pdg=pdgs[1])


points = events.recoParticles.spacePoints[to_merge][eventNum]
startPoints = events.recoParticles.startPos[to_merge][eventNum]
directions = events.recoParticles.direction[to_merge][eventNum]
pdgs = events.trueParticlesBT.pdg[to_merge][eventNum]
beam_mask = np.logical_not(events.recoParticles.number == events.recoParticles.beam_number)[to_merge][q.null]
#* Plot Signal PFOs
if showSignal:
    prod4a_evd.PlotSignalPFO(display, eventNum, signal, points, startPoints, directions, None, 0, -1, False)
    prod4a_evd.PlotSignalPFO(display, eventNum, signal, points, startPoints, directions, None, 1, -1, False)

#* Plot background PFOs
if showBackground: prod4a_evd.PlotBackgroundPFO(display, eventNum, background, beam_mask, points, startPoints, directions, pdg=None, i = -1, plotIP = False)

#* Plot BeamParticle:
beam_mask = events.recoParticles.number == events.recoParticles.beam_number
points = events.recoParticles.spacePoints[beam_mask][eventNum]
pdg = events.trueParticlesBT.pdg[beam_mask][eventNum]
display.PlotPFO(points, marker="o", colour="black", startPoint = events.recoParticles.beamVertex[eventNum], pdg=pdg)

#* Plot beam vertex
display.PlotPoint(events.recoParticles.beamVertex[eventNum], marker="x", colour="red", pointSize=100)

custom_lines = [matplotlib.lines.Line2D([0], [0], color="black", lw=2),
                matplotlib.lines.Line2D([0], [0], color="green", lw=2),
                matplotlib.lines.Line2D([0], [0], color="lime", lw=2),
                matplotlib.lines.Line2D([0], [0], color="blue", lw=2),
                matplotlib.lines.Line2D([0], [0], color="cyan", lw=2),
                matplotlib.lines.Line2D([0], [0], color="orange", lw=2),
                matplotlib.lines.Line2D([0], [0], marker="x", color="red", markersize=15, lw=0),
                ]

display.ax3D.legend(custom_lines, ["beam particle", "start shower 1", "signal 1", "start shower 2", "signal 2", "background", "decay vertex"])
display.xy.legend(custom_lines, ["beam particle", "start shower 1", "signal 1", "start shower 2", "signal 2", "background", "decay vertex"])
display.xy.grid()
display.xz.grid()
plt.tight_layout()
display.DetectorBounds()
plt.show()